# Import lunges for training behavioral classifier

In [1]:
from glob import glob
from ipdb import set_trace
from ipyfilechooser import FileChooser
import pandas as pd
import pickle as pkl
import re

Choose folder with PRHs and lunges in .csv format

In [2]:
fc = FileChooser()
fc.title = '<b>Folder with lunge deployments</b>'
fc.show_only_dirs = True
display(fc)


FileChooser(path='/Users/frank/Documents/GitHub/development/stickleback/notebooks', filename='', title='HTML(v…

List data files and deployment ids.

In [3]:
files = glob(fc.selected_path + "/*")
deployids = set(re.findall(r"[a-z]{2}[0-9]{6}-[0-9]+[a-z]?", ";".join(files)))
lunge_prhs = dict()
lunge_events = dict()

Iterate through deployments and read PRHs, lunges. Save to pickle file.

In [4]:
for deployid in deployids:
    r = re.compile(".*{} 10Hz.*".format(deployid))
    prh_path = list(filter(r.match, files))[0]
    r = re.compile(".*{}_lunge.*".format(deployid))
    lunge_path = list(filter(r.match, files))[0]

    prh0 = pd.read_csv(prh_path)
    prh0.index = pd.to_datetime(prh0["datetime"])
    prh0.drop(["datetime"], axis=1, inplace=True)
    prh0.sort_index(inplace=True)

    fs = "100ms"
    prh_start, prh_end = prh0.index[0].round(fs), prh0.index[-2].round(fs)
    lunge_prhs[deployid] = pd.merge_asof(pd.DataFrame(index=pd.date_range(prh_start, prh_end, freq=fs)), 
                                         prh0,
                                         left_index=True,
                                         right_index=True)

    lunges0 = pd.read_csv(lunge_path)
    lunge_events[deployid] = pd.DatetimeIndex(lunges0["lungetime"])

In [11]:
sizes = [(deployid, len(prh)) for deployid, prh in lunge_prhs.items()]
sizes = sorted(sizes, key = lambda t: t[1])
keep = [t[0] for t in sizes[0:10]]
keep

['bp160614-3b',
 'bp180830-53',
 'bw180827-49',
 'bw180829-42',
 'bw180904-52',
 'mn170220-30',
 'bw180830-48',
 'bb190228-52',
 'bw180827-53',
 'bw180830-52b']

In [12]:
lunge_prhs_subset = {deployid: lunge_prhs[deployid] for deployid in keep}
lunge_events_subset = {deployid: lunge_events[deployid] for deployid in keep}
    
with open("../data/lunges_subset.pkl", "wb") as f:
    pkl.dump((lunge_prhs_subset, lunge_events_subset), f)